# Analyse des données physiques avec XGBoost

Ce notebook implémente la classification XGBoost sur notre jeu de données physiques pour la détection d'attaques. Nous évaluerons les performances à la fois sur la classification binaire (attaque vs normal) et la classification multiclasse (types d'attaques spécifiques).

In [1]:
# Libraries
import pandas as pd
import numpy as np
from pickleshare import PickleShareDB
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score,
    confusion_matrix, matthews_corrcoef, balanced_accuracy_score
)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import tracemalloc
import os
np.random.seed(42)

## Chargement des données

In [2]:
# Load data from prep_data
data_dir = '../prep_data' 
db = PickleShareDB(os.path.join(data_dir, 'kity'))

# Load raw data
df_phy_1 = db['df_phy_1']
df_phy_2 = db['df_phy_2']
df_phy_3 = db['df_phy_3']
df_phy_4 = db['df_phy_4']
df_phy_norm = db['df_phy_norm']

# Load label mapping
label_mapping = db['label_mapping']

## Préparation des données

Pour XGBoost, nous utiliserons la plupart des caractéristiques mais nous pouvons supprimer les colonnes redondantes ou non informatives:

- Supprimer la colonne Time car elle n'est pas pertinente pour la classification
- Garder tous les indicateurs physiques (capteurs de réservoir et de débit)
- Garder les états de vannes et de pompes car ils représentent les actions du système
- Supprimer toutes les colonnes constantes identifiées lors d'analyses précédentes

In [3]:
print("Concatenating datasets...")
df_combined = pd.concat(
    [df_phy_1, df_phy_2, df_phy_3, df_phy_4, df_phy_norm], 
    ignore_index=True
)

y_label_all = df_combined['Label']
y_label_n_all = df_combined['Label_n']
X = df_combined.drop(columns=['Time', 'Label', 'Label_n'])

categorical_cols = X.select_dtypes(include=['category']).columns
X_all = pd.get_dummies(X, columns=categorical_cols)

missing_values = X_all.isnull().sum()
if missing_values.sum() > 0:
    print("Imputing missing values...")
    imputer = SimpleImputer(strategy='mean')
    X_all = imputer.fit_transform(X_all)

# print shape of X and y
print("X shape:", X_all.shape)
print("y shape:", y_label_all.shape)
print("y_n shape:", y_label_n_all.shape)

Concatenating datasets...
X shape: (10923, 26)
y shape: (10923,)
y_n shape: (10923,)


## Implémentation et fonctions d'évaluation XGBoost

In [4]:
def train_evaluate_xgb(X_train, X_test, y_train, y_test):
    """Train and evaluate XGBoost model with performance metrics"""
    print("NaN check in train_evaluate_xgb:")
    print("X_train NaNs:", X_train.isna().sum().sum())
    print("X_test NaNs:", X_test.isna().sum().sum())
    
    # Handle any remaining NaN values if needed
    if X_train.isna().sum().sum() > 0 or X_test.isna().sum().sum() > 0:
        imputer = SimpleImputer(strategy='mean')
        X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
        X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)
    
    xgb = XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    
    # Measure training time and memory
    tracemalloc.start()
    start_fit_time = time.time()
    
    xgb.fit(X_train, y_train)
    
    fit_time = time.time() - start_fit_time
    current, peak = tracemalloc.get_traced_memory()
    fit_memory_usage = peak / (1024 * 1024)  # Convert to MB
    tracemalloc.stop()
    
    # Measure prediction time and memory
    tracemalloc.start()
    start_predict_time = time.time()
    
    y_pred = xgb.predict(X_test)
    
    predict_time = time.time() - start_predict_time
    current, peak = tracemalloc.get_traced_memory()
    predict_memory_usage = peak / (1024 * 1024)  # Convert to MB
    tracemalloc.stop()
    
    # Calculate metrics
    conf_matrix = confusion_matrix(y_test, y_pred)
    TN, FP, FN, TP = conf_matrix.ravel()
    
    return {
        'confusion_matrix': conf_matrix,
        'TP': TP,
        'FP': FP,
        'TN': TN,
        'FN': FN,
        'precision': precision_score(y_test, y_pred, average='binary'),
        'recall': recall_score(y_test, y_pred, average='binary'),
        'tnr': TN / (TN + FP) if (TN + FP) != 0 else 0,
        'fpr': FP / (FP + TN) if (FP + TN) != 0 else 0,
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred, average='binary'),
        'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
        'mcc': matthews_corrcoef(y_test, y_pred),
        'fit_time': fit_time,
        'predict_time': predict_time,
        'fit_memory_usage': fit_memory_usage,
        'predict_memory_usage': predict_memory_usage,
        'feature_importance': dict(zip(X_train.columns, xgb.feature_importances_))
    }

## Entraînement et évaluation du modèle - Classification binaire

In [5]:
# Split data for binary classification
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_label_n_all, test_size=0.2, random_state=42
)

# Train and evaluate model
binary_results = train_evaluate_xgb(X_train, X_test, y_train, y_test)

# Add metadata
binary_results['data'] = 'PHY'
binary_results['model_type'] = 'XGBoost'
binary_results['attack_type'] = 'labeln'

NaN check in train_evaluate_xgb:
X_train NaNs: 0
X_test NaNs: 0


/home/zoe/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [14:06:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


## Visualiser les résultats de la classification binaire

In [6]:
# Plot confusion matrix
fig = go.Figure(data=go.Heatmap(
    z=binary_results['confusion_matrix'],
    x=['Predicted Negative', 'Predicted Positive'],
    y=['Actual Negative', 'Actual Positive'],
    text=binary_results['confusion_matrix'],
    texttemplate="%{text}",
    textfont={"size": 16},
    colorscale='Blues'
))

fig.update_layout(
    title='Confusion Matrix - Binary Classification',
    height=400,
    width=500
)

fig.show()

In [7]:
# Plot feature importance
feature_importance = pd.DataFrame(
    list(binary_results['feature_importance'].items()),
    columns=['Feature', 'Importance']
).sort_values('Importance', ascending=False).head(10)

fig = go.Figure(data=[
    go.Bar(x=feature_importance['Feature'],
           y=feature_importance['Importance'])
])

fig.update_layout(
    title='Top 10 Most Important Features',
    xaxis_title='Feature',
    yaxis_title='Importance Score'
)

fig.show()

## Classification multiclasse

In [8]:
# Encode labels
le = LabelEncoder()
le.classes_ = np.array(list(label_mapping.keys()))
y_label_all_encoded = le.transform(y_label_all)

# Dictionary to store results for each attack type
multiclass_results = {}

# Train and evaluate for each attack type
for attack_label, encoded_label in label_mapping.items():
    if attack_label != 'normal':  # Skip normal class as it's our reference
        print(f"\nProcessing attack type: {attack_label}")
        
        # Create binary labels for this attack type
        y_binary = (y_label_all_encoded == encoded_label).astype(int)
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_all, y_binary, test_size=0.2, random_state=42
        )
        
        results = train_evaluate_xgb(X_train, X_test, y_train, y_test)
        results['data'] = 'PHY'
        results['model_type'] = 'XGBoost'
        results['attack_type'] = attack_label
        multiclass_results[attack_label] = results


Processing attack type: DoS
NaN check in train_evaluate_xgb:
X_train NaNs: 0
X_test NaNs: 0

Processing attack type: MITM
NaN check in train_evaluate_xgb:
X_train NaNs: 0
X_test NaNs: 0


/home/zoe/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning:

[14:06:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/home/zoe/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning:

[14:06:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





Processing attack type: physical fault
NaN check in train_evaluate_xgb:
X_train NaNs: 0
X_test NaNs: 0


/home/zoe/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning:

[14:06:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





Processing attack type: scan
NaN check in train_evaluate_xgb:
X_train NaNs: 0
X_test NaNs: 0


/home/zoe/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning:

[14:06:16] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/home/zoe/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



## Visualiser les résultats de la classification multiclasse

In [9]:
def plot_attack_metrics(metric_name):
    attacks = list(multiclass_results.keys())
    values = [results[metric_name] for results in multiclass_results.values()]
    
    fig = go.Figure(data=[go.Bar(x=attacks, y=values)])
    
    fig.update_layout(
        title=f'{metric_name} by Attack Type',
        yaxis_title='Score',
        xaxis_title='Attack Type'
    )
    return fig

# Plot key metrics
metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1']
for metric in metrics_to_plot:
    plot_attack_metrics(metric).show()

In [10]:
# Plot confusion matrices for each attack type
for attack_type, results in multiclass_results.items():
    fig = go.Figure(data=go.Heatmap(
        z=results['confusion_matrix'],
        x=['Predicted Negative', 'Predicted Positive'],
        y=['Actual Negative', 'Actual Positive'],
        text=results['confusion_matrix'],
        texttemplate="%{text}",
        textfont={"size": 16},
        colorscale='Blues'
    ))
    
    fig.update_layout(
        title=f'Confusion Matrix - {attack_type}',
        height=400,
        width=500
    )
    
    fig.show()

In [11]:
# Create summary table
summary_data = []

for attack_type, results in multiclass_results.items():
    summary_data.append({
        'Attack Type': attack_type,
        'Accuracy': results['accuracy'],
        'Precision': results['precision'],
        'Recall': results['recall'],
        'F1 Score': results['f1'],
        'Training Time (s)': results['fit_time'],
        'Prediction Time (s)': results['predict_time'],
        'Training Memory (MB)': results['fit_memory_usage'],
        'Prediction Memory (MB)': results['predict_memory_usage']
    })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.round(4)
display(summary_df)

,Attack Type,Accuracy,Precision,Recall,F1 Score,Training Time (s),Prediction Time (s),Training Memory (MB),Prediction Memory (MB)
0,DoS,0.9968,0.9839,0.9104,0.9457,0.1758,0.0118,0.9861,0.3220
1,MITM,0.9812,0.9573,0.8220,0.8845,0.0819,0.0144,0.9837,0.3225
2,physical fault,0.9977,0.9855,0.9784,0.9819,2.0785,0.0164,0.9836,0.3210
3,scan,0.9982,0.0000,0.0000,0.0000,0.0802,0.0131,0.9715,0.3013


## Analyse de l'importance des caractéristiques

In [12]:
# Analyze feature importance across different attack types
feature_importance_by_attack = {}

for attack_type, results in multiclass_results.items():
    # Get top 10 features for each attack type
    importance_df = pd.DataFrame(
        list(results['feature_importance'].items()),
        columns=['Feature', 'Importance']
    ).sort_values('Importance', ascending=False).head(10)
    
    feature_importance_by_attack[attack_type] = importance_df
    
    # Plot
    fig = go.Figure(data=[
        go.Bar(x=importance_df['Feature'],
               y=importance_df['Importance'])
    ])

    fig.update_layout(
        title=f'Top 10 Most Important Features for {attack_type}',
        xaxis_title='Feature',
        yaxis_title='Importance Score'
    )

    fig.show()

## Analyse des performances de calcul

In [13]:
# Plot computational performance metrics
performance_fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Training and Prediction Time', 'Memory Usage')
)

# Time metrics
attacks = list(multiclass_results.keys())
train_times = [results['fit_time'] for results in multiclass_results.values()]
predict_times = [results['predict_time'] for results in multiclass_results.values()]

performance_fig.add_trace(
    go.Bar(name='Training Time', x=attacks, y=train_times),
    row=1, col=1
)
performance_fig.add_trace(
    go.Bar(name='Prediction Time', x=attacks, y=predict_times),
    row=1, col=1
)

# Memory metrics
train_memory = [results['fit_memory_usage'] for results in multiclass_results.values()]
predict_memory = [results['predict_memory_usage'] for results in multiclass_results.values()]

performance_fig.add_trace(
    go.Bar(name='Training Memory', x=attacks, y=train_memory),
    row=2, col=1
)
performance_fig.add_trace(
    go.Bar(name='Prediction Memory', x=attacks, y=predict_memory),
    row=2, col=1
)

performance_fig.update_layout(
    height=800,
    title_text="Computational Performance by Attack Type",
    barmode='group'
)

performance_fig.show()

## Sauvegarder les résultats pour Streamlit

In [14]:
# Save binary classification results
db['PHY_results_xgb_labeln'] = binary_results

# Save multiclass results
for attack_label in multiclass_results.keys():
    db[f'PHY_results_xgb_{attack_label}'] = multiclass_results[attack_label]

# Save summary statistics
db['xgb_summary_stats'] = {
    'summary_df': summary_df,
    'binary_results': binary_results,
    'multiclass_results': multiclass_results,
    'feature_importance_by_attack': feature_importance_by_attack
}

## Résumé de l'analyse

Principales conclusions de l'analyse XGBoost :

1. Classification binaire (Attaque vs Normal) :
   - Métriques de performance globales
   - Caractéristiques les plus importantes pour la détection

2. Classification multiclasse (par type d'attaque) :
   - Variations des performances selon les types d'attaques
   - Motifs d'importance des caractéristiques pour différentes attaques

3. Efficacité computationnelle :
   - Analyse du temps d'entraînement et de prédiction
   - Modèles d'utilisation de la mémoire
